In [3]:
#(1) 导入CIFAR-10数据集，分别获得训练集和测试集
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data() # 加载数据，划分测试集和训练集

In [4]:
#（2）对数据进行预处理，将像素值归一化为[0,1],多分类问题将分类标签转化为二元类矩阵
import numpy as np
from tensorflow.keras import utils
from tensorflow.keras.utils import to_categorical

num_classes = 10 # 分类个数
# 转换数据类型
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# 进行归一化处理，因为像素值都是从0-255的
x_train /= 255
x_test /= 255
# 为了将 整数目标值 转换为 分类目标值
# 对每个样本的类元素使用独热编码，将整数转换为10个元素的二进制向量，类值的索引为1。
# 把多分类问题转换成二分类问题
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [6]:
#（3）构建CNN模型
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
model = Sequential() # 定义顺序模型
#第一个卷积块：32个3*3卷积核，步长为1，第一个网络层，需要有输入参数告诉神经网络输入的大小是多少
model.add(Conv2D(32, (3, 3), padding='same',
input_shape=x_train.shape[1:])) # same表示一种补0策略，填充前后图片大小不变
# 添加relu激活函数
model.add(Activation('relu'))
# 32个3*3卷积核
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
# 池化层
model.add(MaxPooling2D(pool_size=(2, 2)))
# 防止过拟合
model.add(Dropout(0.25))
#第二个卷积块
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# 全连接层 # flatten 函数，将图像矩阵展平成一维的向量
model.add(Flatten())
# Dense层 全连接层 512代表该层的输出维度
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
# 最后的Dense层 即全连接层， 输出所有的类别
model.add(Dense(num_classes))
# softmax输出分别属于哪类的概率
model.add(Activation('softmax'))

In [7]:
#（4）对构建好的模型进行编译。这是多分类问题，损失函数选择categorical_crossentrogy，优化器选择RMSprop
from tensorflow import keras
from keras import optimizers

# 初始化RMSprop优化器
# 参考文档https://keras123.com/optimizers/
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# 模型编译
# https://keras.io/zh/losses/
# https://keras.io/zh/models/model/#compile
model.compile(loss='categorical_crossentropy',
optimizer=opt, metrics=['accuracy']) # loss函数，优化目标（多分类常用交叉熵函数）；优化器；评价函数用于评估当前训练模型的性能

In [8]:
#（5）训练CNN模型
# https://keras.io/zh/models/model/#compile
# 批处理,一次训练所抓取的数据样本数量，然后更新权值，减少计算量；
batch_size = 32

# 参数学习算法的迭代次数
epochs = 2 
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
validation_data=(x_test, y_test), shuffle=True) # 是否在每轮迭代之前混洗数据,为了避免数据投入的顺序对网络训练造成影响。

Epoch 1/2
1563/1563 [==============================] - 159s 100ms/step - loss: 1.8461 - accuracy: 0.3239 - val_loss: 1.5491 - val_accuracy: 0.4454
Epoch 2/2
1563/1563 [==============================] - 154s 98ms/step - loss: 1.5210 - accuracy: 0.4480 - val_loss: 1.3932 - val_accuracy: 0.5053


In [9]:
#（6）对CNN模型进行性能评估
# 在测试模式下返回模型的误差值和评估标准值。
# https://keras.io/zh/models/model/#compile
scores = model.evaluate(x_test, y_test, verbose=1)#  0 或 1。日志显示模式。 0 = 安静模式，1 = 进度条。
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 7s 23ms/step - loss: 1.3932 - accuracy: 0.5053
Test loss: 1.3932141065597534
Test accuracy: 0.505299985408783


In [10]:
#（7）使用预训练模型对图像进行分类
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.resnet50 import decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np

model = ResNet50(weights='imagenet')
#导入预训练模型ResNet50
# weights: None 代表随机初始化
# 'imagenet' 代表加载在 ImageNet 上预训练的权值

102981632/102967424 [==============================] - 21s 0us/step


In [12]:
# 对输入图片进行处理
img_path = 'data/elephant.jpg'
# 加载图片，保存在img_path路径下
#  target_size=(224, 224)表示输入的图像尺寸为（224，224），即一幅图像是一个224x224大小的二维数组
img = image.load_img(img_path, target_size=(224, 224))

# 将图像转换为数组
X = image.img_to_array(img)

# expand_dims用来扩展数组维度的，axis=0表示在0的位置加1维
#一般神经网络定义好都是四个维度，所以我们需要再添加一维
X = np.expand_dims(X, axis=0)

# 对样本执行逐样本均值消减 的归一化，去均值中心化，将图像足够适合模型所需的格式
X = preprocess_input(X)

# 模型预测
preds = model.predict(X)
print('Predicted:', decode_predictions(preds, top=3)[0])

49152/35363 [=========================================] - 0s 3us/step
Predicted: [('n01871265', 'tusker', 0.5984406), ('n02504458', 'African_elephant', 0.3467636), ('n02504013', 'Indian_elephant', 0.050883994)]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, image
from tensorflow import keras
from keras import layers
from keras import models
from keras.layers import Dropout
from tensorflow.keras import optimizers
from keras.models import load_model

#输入训练集
train_dir = 'data/training/'
validation_dir = 'data/validation/'
model_file_name = 'cat_dog_model.h5'

#构建初始分类器模型
def init_model():
    model = models.Sequential() #顺序模型
 
    KERNEL_SIZE = (3, 3) #权重是3*3的矩阵
 
    model.add(layers.Conv2D(filters=32, kernel_size=KERNEL_SIZE, activation='relu', input_shape=(150, 150, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
 
    model.add(layers.Conv2D(filters=64, kernel_size=KERNEL_SIZE, activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
 
    model.add(layers.Conv2D(filters=128, kernel_size=KERNEL_SIZE, activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
 
    model.add(layers.Conv2D(filters=128, kernel_size=KERNEL_SIZE, activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
 
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
 
    model.add(Dropout(0.5)) # 采用50%的dropout
 
    model.add(layers.Dense(1, activation='sigmoid'))
 
    model.compile(loss='binary_crossentropy', #loss：定义损失函数
                  optimizer=optimizers.RMSprop(lr=1e-3), #optimizer：指定优化算法
                  metrics=['accuracy']) #metrics：评价指标
  
    return model
 
 
def fig_loss(history):
    history_dict = history.history
    loss_values = history_dict['loss']
    val_loss_values = history_dict['val_loss']
    epochs = range(1, len(loss_values) + 1)
    plt.plot(epochs, loss_values, 'b', label='Training loss')
    plt.plot(epochs, val_loss_values, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()
    plt.show()
 
 
def fig_acc(history):
    history_dict = history.history
    acc = history_dict['accuracy']
    val_acc = history_dict['val_accuracy']
    epochs = range(1, len(acc) + 1)
    plt.plot(epochs, acc, 'g', label='Training acc')
    plt.plot(epochs, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid()
    plt.show()
 
 
def fit(model):
    train_datagen = ImageDataGenerator(rescale=1. / 255)
    validation_datagen = ImageDataGenerator(rescale=1. / 255)
 
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=256,
        class_mode='binary')
 
    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=64,
        class_mode='binary')
 
    history = model.fit_generator(
        train_generator,
        epochs=1,
        validation_data=validation_generator,
    )
 
    model.save(model_file_name)
 
    fig_loss(history)
    fig_acc(history)
 
 
def predict():
    model = load_model(model_file_name)
    print(model.summary())
 
    img_path = './data/test/cat/cat.4021.jpg'
    img = image.load_img(img_path, target_size=(150, 150))
    img_tensor = image.img_to_array(img)
    img_tensor = img_tensor / 255
    img_tensor = np.expand_dims(img_tensor, axis=0)

    plt.imshow(img_tensor[0])
    plt.show()
 
    result = model.predict(img_tensor)
    print(result)
 
 
# 画出count个预测结果和图像
def fig_predict_result(model, count):
    test_datagen = ImageDataGenerator(rescale=1. / 255)
    test_generator = test_datagen.flow_from_directory(
        './data/test/',
        target_size=(150, 150),
        batch_size=256,
        class_mode='binary')
 
    text_labels = []
    plt.figure(figsize=(30, 20))
    # 迭代器可以迭代很多条数据，但我这里只取第一个结果看看
    for batch, label in test_generator:
        pred = model.predict(batch)
        for i in range(count):
            true_reuslt = label[i]
            print(true_reuslt)
            if pred[i] > 0.5:
                text_labels.append('dog')
            else:
                text_labels.append('cat')
 
            # 4列，若干行的图
            plt.subplot(count / 4 + 1, 4, i + 1)
            plt.title('This is a ' + text_labels[i])
            imgplot = plt.imshow(batch[i])
 
        plt.show()
 
        # 可以接着画很多，但是只是随机看看几条结果。所以这里停下来。
        break
 
 
if __name__ == '__main__':
    model = init_model()
    fit(model)
 
    # 利用训练好的模型预测结果。
    predict()
 
    model = load_model(model_file_name)
    #随机查看10个预测结果并画出它们
    fig_predict_result(model, 10)